In [1]:
import numpy as np
import matplotlib.pyplot as plt
import datajoint as dj
import pandas as pd
import pandas as pd
from pathlib import Path

from io_utils import *
%load_ext autoreload
%autoreload 2
dj.blob.use_32bit_dims = True # necessary for pd.blob read

In [2]:
#### MULTI SESSION


animal_id = 'R600'
bdata = dj.create_virtual_module("new_acquisition", "bdatatest")
subject_session_key = {"ratname": animal_id}

pd_blobs = (bdata.Sessions & subject_session_key).fetch(
    "protocol_data", as_dict=True
)

sess_ids, dates, trials = (bdata.Sessions & subject_session_key).fetch(
    "sessid", "sessiondate", "n_done_trials"
)

peh_blobs = (bdata.ParsedEvents * bdata.Sessions & subject_session_key).fetch('peh', as_dict=True)

trial_filter = (trials != 0) & (trials != 1)
pd_blobs, peh_blobs, trials, sess_ids, dates = drop_sessions(
    pd_blobs, peh_blobs, trials, sess_ids, dates, trial_filter
)


pd_dicts = convert_to_dict(pd_blobs)
reformat_pd_dict(pd_dicts)

peh_dicts = convert_to_dict(peh_blobs)

assert len(pd_dicts) == len(peh_dicts) == len(trials) == len(sess_ids) == len(dates) , (
    "number of sessions does not match, assumptions of code below are broken!"
)

[2022-10-21 12:58:50,776][INFO]: Connecting jbreda@datajoint01.pni.princeton.edu:3306
[2022-10-21 12:58:51,068][INFO]: Connected jbreda@datajoint01.pni.princeton.edu:3306


dropping 1 sessions of 89


In [3]:

# #### SINGLE SESSION ####
# session_key = {'sessid': 884257}

# prb = (bdata.Sessions & session_key).fetch(
#     "protocol_data", as_dict=True
# )

# s_id, d, t = (bdata.Sessions & session_key).fetch(
#     "sessid", "sessiondate", "n_done_trials"
# )
# prd = convert_to_dict(prb)
# pd_prepare_dict_for_df(prd)
# session_key = {'sessid': s_id[0]}
# pehb = (bdata.ParsedEvents & session_key).fetch("peh", as_dict=True)
# pehd = bt.transform_blob(pehb[0]['peh'])



In [5]:
def find_and_fix_pd_len_errors(pd_dicts, peh_dicts, sess_ids, trials):

    #TODO: this function is kinda weird and needs to be cleaned into a smaller version
    #TODO i think. could have a fix_within_pd_len_errors fx that wraps
    #TODO and includes the hidden & lenth verifictation

    #TODO then have a function to look at within session trial errors
    #TODO and that reutnrs sessons to drop/edit

    #TODO then you have the drop/fix single trial function all together

    #TODO, finally, i think you need to return the dates? and everything you passed in
    #TODO because of the dropping of sessions
        
    sess_has_extra_trial = np.full((len(sess_ids)), False)
    keep_sess = np.full((len(sess_ids)), True)
    
    for isess, (pd_dict, peh_dict, sess_id, n_trials) in enumerate(zip(
        pd_dicts, peh_dicts, sess_ids, trials)
    ):

        ## WITHIN PD DICT TRIAL LENGTH ERRORS ##
        # check to see if pd is pre HistorySection.m bug fixes
        # where len(each value) was not equal (only applies to DMS/PWM)
        # ! using sa as reference length template- this could lead to 
        # ! errors if sa length was wrong
        if len(pd_dict["sa"]) != len(pd_dict["sb"]):
            _truncate_sb_length(pd_dict)
        if len(pd_dict["sa"]) != len(pd_dict["result"]):
            _fill_result_post_crash(pd_dict)
        
        # verify there are no remaining within pd_dict len errors 
        lens = map(len, pd_dict.values())
        n_unique_lens = len(set(lens))
        assert n_unique_lens == 1, f"length of dict values for {sess_id} unequal!"

        ## PD AND PEH TRIAL LENGTH ERRORS ##
        # sometimes pd has a different amount of trials in a session 
        # then peh and n_completed_trials. In some cases, it's 
        # easy to realign them, in others, the session needs to be
        # dropped from analysis

        pd_trials = len(pd_dict['sides'])
        peh_trials = len(peh_dict)

        assert peh_trials == n_trials, (
            f"""sess {sess_id} peh trials: {peh_trials} 
            does not match n trials: {n_trials}"""
        )

        n_trials_extra = pd_trials - n_trials

        # skip over sessions with matched trials or 9 trials missing.
        # 9 trials extra means runrats froze and tried to restart
        # 10 times before crashing this is marked in pd_dict['results'] 
        # and will be filtered out of the pd df before merging with peh
        if n_trials_extra==0 or n_trials_extra==9:
            continue 
        elif n_trials_extra==1:
            sess_has_extra_trial[isess] = True
            # TODO log print(f"{sess_id} 1 off") 
        else:
            keep_sess[isess] = False
            # TODO log print(f"{sess_id} being dropped bc {n_trials_extra}")
        
    return sess_has_extra_trial, keep_sess

def _truncate_sb_length(protocol_dict):
    """
    Function to correct for bug in HistorySection, see commit:
    https://github.com/Brody-Lab/Protocols/commit/4a2fadb802d64b7ed66891a263a366a8d2580483
    sb vector was 1 greater than n_started_trials due to an
    appending error

    inputs
    ------
    protocol_dict : dict
        dictionary for a single session's protocol_data

    modifies
    -------
    protocol_dict : dict
        updated protocol_dict with sb column length & contents corrected
        if DMS. length only corrected if PWM2 protocol is being used
    """

    # rename for ease
    sa = protocol_dict["sa"]
    sb = protocol_dict["sb"]
    match = protocol_dict["is_match"]

    # if DMS task, can infer values
    if "is_match" in protocol_dict:
        for trial in range(len(sa)):
            if match[trial]:
                sb[trial] = sa[trial]  # update sb
            else:
                assert sa[trial] in MAP_SA_TO_SB, "sa not known"
                sb[trial] = MAP_SA_TO_SB[sa[trial]]
    else:
        print("sb values incorrect, only fixing length")

    sb = sb[0:-1]  # remove extra entry
    protocol_dict["sb"] = sb  # update


def _fill_result_post_crash(protocol_dict):
    """
    Function to correct for bug in HistorySection, see commit:
    https://github.com/Brody-Lab/Protocols/commit/3bdde4377ffde011cc34d098acfeb77b74c9e606
    result vector was shorter than n_started_trials because program
    crashed & results_history vector was not being properly filled
    during crash clean up

    inputs
    ------
    protocol_dict : dict
        dictionary for a single session's protocol_data

    modifies
    -------
    protocol_dict : dict
        updated protocol_dict with results column length corrected to
        reflect crash trials
    """

    # rename for ease
    results = protocol_dict["result"]
    sa = protocol_dict["sa"]

    # pack with crash result value (5)
    crash_appended_results = np.ones((len(sa))) * 5
    crash_appended_results[0 : len(results)] = results

    protocol_dict["result"] = crash_appended_results

In [7]:


sess_has_extra_trial, keep_sess = find_and_fix_pd_len_errors(pd_dicts, peh_dicts, sess_ids, trials)
pd_dicts, peh_dicts, trials, sess_ids, dates = drop_sessions(
    pd_dicts, peh_dicts, trials, sess_ids, dates, keep_sess
)
drop_extra_trial(pd_dicts, sess_has_extra_trial, trials)

x1, x2 = find_and_fix_pd_len_errors(pd_dicts, peh_dicts, sess_ids, trials)

assert (np.sum(x1) == 0) and (np.sum(x2) == len(peh_dicts)), "lengths not right still!"

dropping 0 sessions of 88
